Scott Broderick, An Su, and Krishna Rajan<Br>
Department of Materials Design and Innovation<Br>
University at Buffalo

# Principal Component Analysis (PCA)

The primary equation for PCA is:

<pre> X = T*P<sup>T</sup> + E                                                     [1] </pre>

Where X is the input data, T is the scores matrix, P is the loadings matrix, and E is the residual matrix. It seems quite simple. But what do these matrices really contain?
Let’s do an example, with the following input data:

In [13]:
# We use Plotly, a Python graphing library, to help us plot nice graphs and tables
# If you are interested in Plotly, please visit https://plot.ly/python/getting-started/
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Pandas and Numpy are used to manipulate the data
import pandas as pd
import numpy as np

# Initiate the Plotly Notebook mode
init_notebook_mode(connected=True)

# Define a plot function to print a table conveniently
def plot_df(df):
    trace = go.Table(header = dict(values = list(df.reset_index().columns),
                        fill = dict(color='#C2D4FF'),
                        align = ['left'] * 5),    
                    cells = dict(values = [df.index,
                        df['Electrical Conductivity (10^6/cm ohm)'],
                        df['Thermal Conductivity at 300K (W/cmK)'],
                        df['Melting Point (K)'],
                        df['Atomic Weight'],
                        df['Density @ 293K (g/cm3)']],
                        fill = dict(color='#F5F8FF'),
                        align = ['left'] * 5))
    return [trace] 

# Create a Pandas DataFrame for input data
data = [[0.02,0.16,1814,44.96,2.99],
        [0.02,0.22,1933,47.88,4.54],
        [0.05,0.31,2160,50.94,6.11],
        [0.08,0.94,2130,52.00,7.19]]
input_df = pd.DataFrame(data,index = ['Sc','Ti','V','Cr'],columns=['Electrical Conductivity (10^6/cm ohm)',
                                'Thermal Conductivity at 300K (W/cmK)', 
                                'Melting Point (K)',
                                'Atomic Weight',
                                'Density @ 293K (g/cm3)'])
input_df.index = input_df.index.set_names(['Element'])

# Plot the input data
iplot(plot_df(input_df), filename = 'Input_Data_table')


## Step 1: What is X?

Since this is the input data, is it X from equation 1? No. An important consideration of X is that it is the pre-processed input data. Aside 1: in singular value decomposition (SVD) – another common data mining method – this data is indeed X in equation 1. The lack of removing the mean of the variables from the data as pre-processing is the primary difference between PCA and SVD. Aside 2: When do we not want to remove the mean? If we are interested in doing a study of the background as background effects are reduced in PCA, or if we are dealing with sparse data, where the measure of the mean of the data is not reliable.

In PCA, the rows of this matrix, which we will term X<Sub>input</Sub>, are the conditions (i.e. what we can control – chemistry, processing, structure, etc.) while the columns are the responses (e.g. properties, measurements, etc.). Note, if you do not pre-process the data such as in SVD, the final analysis result of the transpose is identical, and just T and P are opposite. If you do pre-process the data, then conditions must be the rows and responses must be the columns, otherwise your answer will be wrong.

First, we will subtract the mean (average) of each column from X<sub>input</sub>. The reason for removing the mean is so we can calculate the relational change in responses as a function of changing conditions. That is, without mean centering, we can identify general trends in the data but cannot easily capture specific relationships in the data. For example, melting point increases along with row number, but withoutmean centering, defining the physical explanation becomes challenging. This will become clearer when interpreting the final results. Subtracting the means of X<Sub>input</Sub> from each column yields X<sub>mean</sub> of:

In [14]:
# Calculate Xmean
Xmean = input_df - input_df.mean()
Xmean.loc['Average'] = Xmean.mean()

# Plot Xmean
iplot(plot_df(Xmean.round(4)), filename = 'Xmean_table')

Note that after mean centering, the mean of each column is zero. One possible error is the number of places beyond the decimal point to include. If you get average values that are not exactly zero, this error will be carried out through the entire calculation, so if you are not getting zero, check that you are consistent in the number of decimal points used throughout the analysis. Also, in cases where there are obvious outliers in the data, using the median instead of the mean may be appropriate to reduce the effect of outliers.

After mean centering the data, we standardize the data. The reason is because the units of the variables are different. If instead of using K for melting point we instead used mK, the final result would significantly change, although the data is the same. Additionally, melting point values have magnitude of 10<sup>3</sup>, while electrical conductivity has magnitude of 10<sup>-2</sup> so that equal percent changes between columns results in very different quantitative changes. In cases where the units are the same (for example, spectral data or repeated experiments), standardization may not be necessary.

An issue is whether to use the sample standard deviation or the population standard deviation. The values will be different because for the sample standard deviation, the deviation in the data is divided by degrees of freedom instead of the number of samples as in population standard deviation. Logically, we would use the sample standard deviation because we are not using a complete data set; that is, the data set we have could be expanded to a larger number of elements. Thus, in order for the results to be more robust, we will use the sample standard deviation.

The matrix below is X in equation 1.

In [15]:
# Calculate X 
X = (input_df - input_df.mean())/input_df.std()

# Plot X
iplot(plot_df(X.round(4)), filename = 'X_table')

## Step 2: What is P?

P, the loadings matrix, describes the correlation between the responses (columns of X). To calculate P, we first must calculate the correlation matrix of X. However, the calculation of the correlation matrix is only correct when using the population standard deviation. In the prior section, we calculated X using the sample standard deviation, as we have not collected all possible data. However, for calculating the correlation in the data, the assumption must be made that the correlation is being calculated on the entire data. Therefore, we define X<sub>Pop</sub>:

In [16]:
# Plot XPop
Xpop = (input_df - input_df.mean())/input_df.std(ddof = 0)
iplot(plot_df(Xpop.round(4)), filename = 'pandas_table4')

The correlation matrix is calculated for X<sub>Pop</sub>. If correlation is calculated on X instead of X<sub>Pop</sub>, the correlation matrix will have diagonal values not equal to unity, which is a requirement of the correlation matrix. The correlation matrix describes how variables change with respect to other variables (hence covariance instead of variance). The correlation between variables x and y can be thought of as the slope of a graph of x vs. y. If x=y then correlation = 1, if x=-y, then correlation = -1, if x=2y, then correlation = 0.5, and so on. This interpretation of correlation values will be useful when we interpret our final PCA results. Note, if you plot the variables of X versus each other, the points do not fall completely on a single line, and therefore the slope does not capture 100% of the correlation in the data. We will discuss how this impacts our analysis later on.

The correlation matrix (which we will term C<sub>X</sub>) of X<sub>Pop</sub> is as follows:

In [17]:
# Calculate Cx
Cx = Xpop.corr()

# Plot Cx
iplot(plot_df(Cx.round(4)), filename = 'Correlation matrix table')

Note that the diagonal of C<sub>X</sub> is 1 (corr(x,x)=1). This is obvious as every point of a graph of x vs. x will fall exactly on a line of slope 1. If you do not have diagonal values of 1, then you have an error, and this error will be carried out through your analysis. A possible reason for this error is that the correlation is being calculated on a different matrix than X<sub>Pop</sub>, such as X, X<sub>input</sub> or X<sub>mean</sub>. Again, the reason for using the population standard deviation is that we make the assumption that the data in the matrix contains all possible data, as opposed to a representation of a limited portion. This assumption is necessary for this step calculation to perform.
In all cases, corr(x,y) = corr(y,x). The reason is because:

<Pre> Corr(x,y) = function[(X<sub>i</sub>-X<sub>mean</sub>)(Y<sub>i</sub>-Y<sub>mean</sub>)]                                               [2] </Pre>

Therefore, switching X and Y makes no difference, as X\*Y = Y\*X. 

Note the dimensions of the matrix. While the dimensions for X were number of conditions and number of responses, the dimensions of Cx are number of conditions and number of conditions. This is important because the correlation matrix must be a square matrix. We will next take the eigenvalues and eigenvectors of this matrix, which is only possible with a square matrix.

Eigenvectors are vectors of a matrix which remain the same after a transformation, while eigenvalues are the scaling of the vector after the transformation. We will not describe eigenvectors and eigenvalues further here, but will describe how to calculate them from the correlation matrix. You can easily extract eigenvalues and eigenvectors from numerous programs or online applets, but we will describe the logic in terms of matrices here.

First, we will identify the eigenvalues of the correlation matrix. The number of eigenvalues is equal to the number of dimensions (in this example there are 5 eigenvalues). For the moment, we will calculate all five eigenvalues, but we will not necessarily use them all later on. We define the identity matrix (I), which has the same columns and rows as C<sub>X</sub>, and has diagonal values of 1, and all other values as 0.

The eigenvalues are then defined as the values of λ for which the determinant of C<sub>X</sub>- λI =0. The eigenvector vi is then defined as the vector which makes the equation:<br>
<pre>(C<sub>X</sub>- λ<sub>i</sub>I)*v<sub>i</sub> = 0                                                          [3] </pre>
        

Using Numpy's linalg function, we find the five eigenvalues (Determinant=0) as well as the eigenvectors corresponding to the eigenvalues. Note: in this, the rows correspond to the columns of X, while the columns are the different PCs, ordered by eigenvalue. This matrix of the eigenvectors, with the order of PCs determined by the eigenvalues, is matrix P in equation 1.

In [18]:
eig_vals, eig_vecs = np.linalg.eig(Cx)

P = pd.DataFrame(eig_vecs, index = ['Electrical Conductivity (10^6/cm ohm)',
                                'Thermal Conductivity at 300K (W/cmK)', 
                                'Melting Point (K)',
                                'Atomic Weight',
                                'Density @ 293K (g/cm3)'], columns = ['PC1','PC2','PC3','PC4','PC5'])
P_round = P.round(3)

trace = go.Table(header = dict(values = list(P_round.reset_index().columns),
                        fill = dict(color='#C2D4FF'),
                        align = ['left'] * 5),    
                    cells = dict(values = [P_round.index,
                        P_round['PC1'],
                        P_round['PC2'],
                        P_round['PC3'],
                        P_round['PC4'],
                        P_round['PC5']],
                        fill = dict(color='#F5F8FF'),
                        align = ['left'] * 5))

iplot([trace], filename = 'eigen_vectors_table')

In [19]:
# Plot the eigenvalues 
# Source code: https://plot.ly/ipython-notebooks/principal-component-analysis/
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
np.set_printoptions(precision =3, suppress = True)
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print("%.3f"%i[0])

Eigenvalues in descending order:
4.493
0.459
0.048
0.000
0.000


Note, there are only three non-zero eigenvalues. Each eigenvalue, and the corresponding eigenvector, represent a Principal Component (PC). We rank the eignvalues in terms of value, so that the largest eigenvalue corresponds to PC1, the second largest eigenvalue corresponds to PC2, and so on. So effectively, for this data there are only three PCs. The explained variance captured by different PC is equal to that respective eigenvalue divided by the sum of all eigenvalues. 

In [20]:
# Plot the explained variance captured by different PC
# Source code: https://plot.ly/ipython-notebooks/principal-component-analysis/
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
var_exp_round = np.around(var_exp, decimals = 2)
cum_var_exp = np.cumsum(var_exp)
cum_var_exp_round = np.around(cum_var_exp, decimals = 2)

trace1 = dict(
    type='scatter',
    x=['PC %s' %i for i in range(1,6)],
    y=var_exp_round,
    name='Individual'
)

trace2 = dict(
    type='scatter',
    x=['PC %s' %i for i in range(1,6)], 
    y=cum_var_exp_round,
    name='Cumulative'
)
data = [trace1, trace2]

layout=dict(
    title='Explained variance by different principal components',
    yaxis=dict(
        title='Explained variance in percent'
    ),
    annotations=list([
        dict(
            x=1.16,
            y=1.05,
            xref='paper',
            yref='paper',
            text='Explained Variance',
            showarrow=False,
        )
    ])
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='selecting-principal-components')

Thus, for this data PC1 captures 89.86% of the variance of the data and PC1+PC2 capture 99.04% of the variance. The use of PCA for dimensionality reduction now becomes clear, as three PCs capture the same information as the five dimensions of X<Sub>input</Sub>. The plot of eigenvalue versus PC is referred to as a ‘scree plot’. This plot will become useful in the upcoming section “What is E?”.

## Step 3: What is T?

To understand the calculation of T, we first must better understand the loadings matrix (P). P defines a series of new axes which are a linear combination of the variables, ranked to maximize the information captured. Each axis is then defined as the variables times the eigenvectors, so that the first axis is v1 times variables. Thus the first axis capturing 89.86% of the information and second axis capturing 9.18% of the variance in the data is: 

PC1 = -0.455 Electrical cond - 0.412 Thermal cond – 0.438 T<sub>m</sub> - 0.461 At. Wt. - 0.468 Density 
 
PC2 = -0.313 Electrical cond - 0.712 Thermal cond + 0.538 T<sub>m</sub> + 0.295 At. Wt. + 0.137 Density 

The scores matrix then becomes the plotting of the original data in this new space. As opposed to the normal plotting of variable 1 vs. variable 2, which captures a proportional amount of variance in the data, we now plot a combination of all variables vs. another combination of all variables, which captures a very significant amount of the variance in the data. The scores matrix (T) is then simply the plotting of the variable data (X) times the loadings matrix (P). This is clear from equation 1, with us ignoring E for the moment. The reason we are multiplying X and P is that equation 1 uses the transpose of P, and thus dividing X by the transpose of P is equivalent to multiplying X and P. Note, we are using X as defined in step 1, and not X<sub>Pop</sub> which we used in step 2.

In [21]:
T = X.dot(P)
T_round = T.round(3)

trace = go.Table(header = dict(values = list(T_round.reset_index().columns),
                        fill = dict(color='#C2D4FF'),
                        align = ['left'] * 5),    
                        cells = dict(values = [T_round.index,
                                           T_round['PC1'],
                                           T_round['PC2'],
                                           T_round['PC3'],
                                           T_round['PC4'],
                                           T_round['PC5']],
                        fill = dict(color='#F5F8FF'),
                        align = ['left'] * 5))
iplot([trace], filename = 'T_table')



The matrix above is then T in equation 1.

If you are not getting these values, verify that you are using the preprocessed data X, as opposed to the non-standardized data. Note that the dimensions of this matrix are the conditions versus the number of PCs, while P had dimensions of responses versus number of PCs. Therefore, T*P<sup>T</sup> has dimensions of conditions versus responses, which matches the dimensions of X. The number of columns (PCs) for both T and P must be the same. The columns of T and P are important when we interpret the results. The values in this matrix T, are the variable values for these new variables.

It is possible that you can have the opposite signs in some rows or columns of P and T. As we will discuss in the interpretation section, the results are considered relative. Therefore, if every entry in a column of P is multiplied by negative 1, the interpretations will not have changed, assuming that the signs of that column in T also changed sign accordingly.

## Step 4: What is E?

Up to this point, we have ignored the residual matrix (E). Often in PC analyses, there is no need to define E, and to define T and P as we have here. However, in noise analyses or noise reduction, uncertainty analyses, or error measurements, the definition of E is important. Additionally, when advancing to predictive techniques such as partial least squares (PLS) which build on PCA, the definition of E is critical. To calculate E, we first must revisit the eigenvalues and skree plot that we calculated in step 2.

As discussed, the number of PCs is equal to the number of variables. However, not all PCs provide useful information. We will now select how many PCs are physically significant. There are two general rules-of-thumb for selecting the number of PCs. First, is to select the number of PCs such as that greater than 90% of the variance of the data is captured. Second, is to select the PCs up to the elbow of the scree plot. That is, to select a value of PC up until the scree plot becomes significantly more horizontal. Beyond this point, the data becomes primarily noise. As an aside, this separation of noise has significant implications for PLS then because the regressions are being performed on only physically significant data.

Based on both of these criteria, we identify two PCs as the optimal number. Thus, the scores (T) and loadings (P) matrices will include only two PCs, instead of five as we previously showed. The final scores and loadings matrices are thus shown as follows:

In [22]:
T = T.drop(columns = ['PC3','PC4','PC5'])
T_round = T.round(3)
trace = go.Table(header = dict(values = list(T_round.reset_index().columns),
                        fill = dict(color='#C2D4FF'),
                        align = ['left'] * 5),    
                        cells = dict(values = [T_round.index,
                                           T_round['PC1'],
                                           T_round['PC2']],
                        fill = dict(color='#F5F8FF'),
                        align = ['left'] * 5))
iplot([trace], filename = 'T_table_final')

In [23]:
P = P.drop(columns=['PC3','PC4','PC5'])
P_round = P.round(3)
trace = go.Table(header = dict(values = list(P_round.reset_index().columns),
                        fill = dict(color='#C2D4FF'),
                        align = ['left'] * 5),    
                        cells = dict(values = [P_round.index,
                                           P_round['PC1'],
                                           P_round['PC2']],
                        fill = dict(color='#F5F8FF'),
                        align = ['left'] * 5))
iplot([trace], filename = 'P_table_final')


Having final matrices of T and P, we can now calculate E using equation 1. E is equal to the standardized data matrix (X) minus these final T and P matrices. The resulting matrix of E is then:

In [24]:
E = X - np.dot(T,P.transpose())
iplot(plot_df(E.round(3)), filename = 'E_table')

## Step 5: Interpretation of PC Matrices

The most convenient way for interpreting our results is to plot both the scores and loading matrix in PC1 vs. PC2 space. First, we will look at the loading plot. In these plots, degree of correlation is defined by proximity between points. Additionally, we can define correlation as follows:

<Pre>Correlation between var1 and var2 = cos (angle var1-origin-var2)               [4]</Pre>

Thus, points in opposite quadrants are inversely correlated, points in the same quadrant are correlated, while the correlation between points in adjacent quadrants is less clear. For the loadings plot of our analysis (below), none of the variables are inversely correlated, as there are no positive PC1 values. We find density, atomic weight and melting point are correlated, electrical and thermal conductivity are correlated, and the relationship between atomic weight, density and melting point with electrical and thermal conductivities is more complex.


In [25]:
trace1 = dict(
    type='scatter',
    x=P_round['PC1'],
    y=P_round['PC2'],
    text = ['Electrical Conductivity','Thermal Conductivity','Melting Point','Atomic Weight','Density'],
    mode='markers+text',
    textposition='bottom center'
)

data = [trace1]

layout=dict(
    title='Loading Plot of PCA',
    xaxis=dict(
        title='Loadings on PC1 (89.86%)',
        range = [-0.48, -0.4]
    ),
    yaxis=dict(
        title='Loadings on PC2 (9.18%)',
    )
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='Loading Plot')

The strongest correlation in all of the data is between atomic weight and density as they sit closest together. This correlation makes physical sense, as we can define density as follows:

<Pre>density = mass / volume = (mass/atom) * (atom/volume)                                  [5]</Pre>

Therefore, we would expect a strong correlation between density and atomic weight (i.e. mass/atom). The term atom/volume has primarily to do with attraction between atoms, but in the case of transition metals of consecutive atomic number, we would anticipate that the differences in atom/volume would be minor. Therefore, this finding is logical. Similarly, the correlation between electrical conductivity and thermal conductivity can be explained by the Wiedemann-Franz law, which identifies thermal conductivity to be a function of electrical conductivity.

The measure of correlation between variables as described in equation 4 should be related to the correlation matrix. Looking at the correlation matrix we calculated earlier, we found that the highest value was 0.99 between atomic weight and density. Therefore, based on the mathematical criteria as well as the physical explanation discussed, our results here are validated.

We next look at the scores plot. In this plot, the different elements are plotted in the axes defined by the loadings plot. In this case, we cannot make comparisons in correlation based on quadrants (equation 4) because we mean centered the data. If the data is not mean centered, then the scores can be interpreted like the loadings plot. As this data was mean centered, we are only interested in magnitude of distance between points. In this scores plot, we identify that PC1 is capturing trends related with atomic number, because as atomic number increases, PC1 decreases. It is logical that this relationship would exist as PC1 captures the largest variance in the data and many properties change with atomic number. Note however that we did not input atomic number into the analysis, and so capturing trends between atomic number and the input variables is the type of information used to develop design rules. All of the variables have a positive relationship with atomic number as they all had negative PC1 values, and PC1 decreases with increasing atomic number. In this example the relationship is obvious, but in larger data sets, extracting these type of “hidden” relationships is very important.

In [26]:
trace1 = dict(
    type='scatter',
    x=T_round['PC1'],
    y=T_round['PC2'],
    text = T_round.index,
    mode='markers+text',
    textposition='bottom center'
)

data = [trace1]

layout=dict(
    title='Scores Plot of PCA',
    xaxis=dict(
        title='PC1 (89.86%)',
        range = [-3, 3]
    ),
    yaxis=dict(
        title='PC2 (9.18%)',
        range = [-1, 1]
    )
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='Loading Plot')

Step 6: Expanding to Predictive Approaches

Understanding PCA is necessary to understand PLS, which is a predictive technique as opposed to a classification or dimensionality reduction technique. The fundamental mathematical equations of PLS are:

<Pre>X = T*P<Sup>T</Sup> + E</Pre>

<Pre>Y = U*C<Sup>T</Sup> + F                                                              [6]</Pre>

where both equations are identical to the mathematics we have discussed here. X is the predictor variables, that is what is known for all the systems, while Y is the predicted variable, that is what we want to predict. U, C and F are the scores, loadings, and residual of Y, respectively. These equations are then combined so that:

<Pre>Y = X\*P\*C<Sup>T</Sup> + F                                                           [7]</Pre>

Clearly PLS is based on the PCA mathematics. Additionally, the selection of PCs to include in E significantly impacts the results, as the data included in T*P<Sup>T</Sup> is used in the prediction, while E is not. PLS thus acts a regression within PC spaces.